In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install sentencepiece
!pip install datasets

In [ ]:
from pathlib import Path
# モデルのディレクトリ
# strで指定すると、from_pretrainedでなぜかHuggingFaceを参照したため、PathLikeで指定した。
output = Path("/content/drive/MyDrive/ColabNotebooks/line_to_gpt2/output/")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-small")
tokenizer.do_lower_case = True
model = AutoModelForCausalLM.from_pretrained(output)
model.to(device)
model.eval()

In [ ]:
def generate_reply(inp, num_gen=1):
    input_text = "<s>" + str(inp) + "[SEP]"
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    out = model.generate(input_ids, do_sample=True, max_length=128, num_return_sequences=num_gen
                         top_p=0.95, top_k=50, bad_words_ids=[[1], [5]], no_repeat_ngram_size=3)

    print(">", "あなた")
    print(inp)
    print(">", "Y君")
    for sent in tokenizer.batch_decode(out):
        sent = sent.split('[SEP]</s>')[1]
        sent = sent.replace('</s>', '')
        sent = sent.replace('<br>', '\n')
        print(sent)

In [ ]:
msg = "おはよー"
generate_reply(msg)